In [1]:
import os

In [2]:
%pwd

'd:\\MyDrive\\Personal Data\\End-to-End-Machine-Learning-Project-with-MLFlow\\research'

In [3]:
os.chdir("../")

In [5]:
%pwd

'd:\\MyDrive\\Personal Data\\End-to-End-Machine-Learning-Project-with-MLFlow'

In [6]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Shoaib-Alauudin/End-to-End-Machine-Learning-Project-with-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Shoaib-Alauudin"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f42b27e81f01196ca6c58b286c047cc1585ebfd4"

### entity/config_entity.py

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

### Configuration

In [8]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Shoaib-Alauudin/End-to-End-Machine-Learning-Project-with-MLFlow.mlflow",
        )

        return model_evaluation_config

#### Components

In [26]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from MLProject.entity.config_entity import ModelEvaluationConfig
from MLProject.utils.common import save_json

In [27]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = np.abs(mean_absolute_error(actual, pred))
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Store metrics as a .json file locally
            scores = {'rmse': rmse,
                      'mae': mae,
                      'r2': r2}
            
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry does not work with the file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
                
            else:
                mlflow.sklearn.log_model(model, "model")

#### Pipeline

In [28]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config= model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2023-08-14 01:05:20,839: INFO: common :yaml file: config\config.yaml loaded successfully]
[2023-08-14 01:05:20,849: INFO: common :yaml file: params.yaml loaded successfully]
[2023-08-14 01:05:20,852: INFO: common :yaml file: schema.yaml loaded successfully]
[2023-08-14 01:05:20,853: INFO: common :created directory at: artifacts]
[2023-08-14 01:05:20,855: INFO: common :created directory at: artifacts/model_evaluation]
[2023-08-14 01:05:21,285: INFO: common :json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2023/08/14 01:05:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 2
Created version '2' of model 'ElasticNetModel'.


#### Re-Execute the above code After change the Params.yaml values (alpha: 0.9 & l1_ratio:0.8)